In [ ]:
#works with cv virtual environment on piggy / tf_2 on janice

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os import system
import os
import random

import imageio

import sys
import glob
# import cv2

import pickle

In [2]:
# import sys
# import glob
# import os
# import cv2
# import numpy as np

# from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D
# from keras.models import Model, load_model
# from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
# from keras import backend as K
# import keras

In [16]:
# folder_path = '/home/niaki/Code/hpatches/hpatches-benchmark/data/hpatches-release'  # piggy
# folder_path = '/scratch/hpatches/hpatches-benchmark/data/hpatches-release'  # janice
folder_path = 'hpatches_seqs/'  # janice
patch_size = 65

In [17]:
# all types of patches 
tps = ['ref','e1','e2','e3','e4','e5','h1','h2','h3','h4','h5',\
       't1','t2','t3','t4','t5']

class hpatches_sequence:
    """Class for loading an HPatches sequence from a sequence folder"""
    itr = tps
    def __init__(self,base):
        name = base.split('/')
        self.name = name[-1]
        self.base = base
        for t in self.itr:
            im_path = os.path.join(base, t+'.png')
#             im = cv2.imread(im_path,0)
#             print(im.shape)
#             print(im[0,0])
#             print(type(im[0,0]))
            im = imageio.imread(im_path)
#             print(im_imageio.shape)
#             print(im_imageio[0,0])
#             print(type(im_imageio[0,0]))
#             print(np.array_equal(im, im_imageio))
#             print()            
            self.N = im.shape[0]//patch_size
            setattr(self, t, np.split(im, self.N))

In [18]:
seqs = glob.glob(folder_path+'/*')
seqs = [os.path.abspath(p) for p in seqs]

In [19]:
len(seqs)

3

In [10]:
# def unpickle_vars(pickle_file_path):
#     try:
#         variables = pickle.load(open(pickle_file_path, "rb"))
#         return variables
#     except Exception as e:
#         print("Problem while trying to unpickle: ", str(e))
#         return None

In [20]:
descr_name = 'vae_0.0.0.4'
base_dir = '/home/niaki/Code/sundry_segments/weights'
model_version = '0.0.0.4_dataaug_300moreepoch'
vae_patch_size = 56

In [21]:
latent_dim = 128

In [22]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [23]:
# encoder_weights = unpickle_vars('zimnica/pickled_vae_encoder_0.0.0.4_20200713_114306.pickle')

In [24]:
# decoder_weghts = unpickle_vars('zimnica/pickled_vae_decoder_0.0.0.4_20200713_114306.pickle')

In [25]:
# init_kernel = encoder_weights[0]
# init_bias = encoder_weights[1]
# kernel_initializer = tf.keras.initializers.constant(init_kernel)
# bias_initializer = tf.keras.initializers.constant(init_bias)

In [26]:
class Encoder(layers.Layer):
    """Maps input patches to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        
        self.conv1 = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")
        self.conv2 = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")
        self.conv3 = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(256, activation="relu")
        self.dense_mean = layers.Dense(latent_dim, name="z_mean")
        self.dense_log_var = layers.Dense(latent_dim, name="z_log_var")
        self.sampling = Sampling()
        

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense(x)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))        
        return z_mean, z_log_var, z

In [27]:
class Decoder(layers.Layer):
    """Converts z, the encoded patch vector, back into the patch."""

    def __init__(self, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        
        self.dense = layers.Dense(7 * 7 * 64, activation="relu")
        self.reshape = layers.Reshape((7, 7, 64))
        self.conv1 = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")
        self.conv2 = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")
        self.conv3 = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")
        self.conv_output = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")
        
    def call(self, inputs):
        x = self.dense(inputs)
        x = self.reshape(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv_output(x)
        return x

In [28]:
class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        name="vae",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.encoder = Encoder()
        self.decoder = Decoder()

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

In [29]:
vae = VariationalAutoEncoder()

In [30]:
# have to do this in order to be able to load_weights into vae, see:
# https://github.com/keras-team/keras/issues/10417
fake_batch = np.ones((5, 56, 56, 1), dtype=np.float32)
fake_batch_reconstructed = vae(fake_batch)

In [31]:
vae.load_weights('/scratch/image_datasets/2_for_learned_brief/ready/vae_' + model_version + '.h5')

In [ ]:
# for seq_path in seqs:
#     seq = hpatches_sequence(seq_path)
#     path = os.path.join(descr_name,seq.name)
#     if not os.path.exists(path):
#         os.makedirs(path)
#     descr = np.zeros((seq.N,2)) # trivial (mi,sigma) descriptor
#     for tp in tps:
#         print(seq.name+'/'+tp)
#         for i,patch in enumerate(getattr(seq, tp)):
#             mi = np.mean(patch) # trivial (mi,sigma) descriptor
#             sigma = np.std(patch) # trivial (mi,sigma) descriptor
#             descr[i] = np.array([mi,sigma])
#         np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

In [32]:
for seq_path in seqs:
    seq = hpatches_sequence(seq_path)
    path = os.path.join("hpatches_extracted_descrs", descr_name, seq.name)
    if not os.path.exists(path):
        os.makedirs(path)

    descr_size_per_patch = 128
    descr = np.zeros((seq.N,descr_size_per_patch))
#     print(seq.name)
    for tp in tps:
        print(seq.name+'/'+tp)
        for i,patch in enumerate(getattr(seq, tp)):
            patch = patch / 255.0
#             print(patch)
            patch_crop = patch[4: 60, 4: 60]
            patch_crop_encoded_zzz = vae.encoder(np.expand_dims(np.expand_dims(patch_crop, axis=-1), axis=0))
            patch_crop_encoded_zzz = np.array(patch_crop_encoded_zzz)
            patch_crop_encoded = patch_crop_encoded_zzz[2]
            patch_crop_encoded_flat = patch_crop_encoded_zzz[2].flatten()  
#             print(patch_crop.shape)
#             print(np.array(patch_crop_encoded).shape)
#             print(patch_crop_encoded_flat.shape)
#             print(patch_crop_encoded_flat)
#             print()
            descr[i] = patch_crop_encoded_flat
        np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',')

i_nijmegen/ref

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



KeyboardInterrupt: 

In [34]:
input_shape = (56, 56, 1)
input_img = layers.Input(shape=input_shape)

x = layers.Conv2D(32, (3, 3), activation="relu", padding="valid")(input_img)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="valid")(x)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="valid")(x)
x = layers.Flatten(data_format="channels_last")(x)
x = layers.Dense(256, activation="relu")(x)
dense_mean = layers.Dense(latent_dim, name="z_mean")(x)
dense_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
sampling = Sampling()(dense_mean, dense_log_var)

vae_encoder = Model(input_img, sampling)
vae_encoder.summary()


TypeError: in converted code:


    TypeError: tf__call() takes 2 positional arguments but 3 were given


In [45]:
encoder_inputs = keras.Input(shape=(56, 56, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
vae_encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
vae_encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 56, 56, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 28, 28, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 14, 14, 64)   18496       conv2d_12[0][0]                  
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 7, 7, 64)     36928       conv2d_13[0][0]                  
____________________________________________________________________________________________

In [46]:
print(vae_encoder.get_layer(index=0))

print(vae_encoder.get_layer(index=1))
vae_encoder.get_layer(index=1).set_weights(vae.encoder.conv1.get_weights())

print(vae_encoder.get_layer(index=2))
vae_encoder.get_layer(index=2).set_weights(vae.encoder.conv2.get_weights())

print(vae_encoder.get_layer(index=3))
vae_encoder.get_layer(index=3).set_weights(vae.encoder.conv3.get_weights())

print(vae_encoder.get_layer(index=4))

print(vae_encoder.get_layer(index=5))
vae_encoder.get_layer(index=5).set_weights(vae.encoder.dense.get_weights())

print(vae_encoder.get_layer(index=6))
vae_encoder.get_layer(index=6).set_weights(vae.encoder.dense_mean.get_weights())

print(vae_encoder.get_layer(index=7))
vae_encoder.get_layer(index=7).set_weights(vae.encoder.dense_log_var.get_weights())

print(vae_encoder.get_layer(index=8))

In [47]:
patch_crop_encoded_zzz = vae.encoder(np.expand_dims(np.expand_dims(patch_crop, axis=-1), axis=0))

In [48]:
patch_crop_encoded_zzz2 = vae_encoder.predict(np.expand_dims(np.expand_dims(patch_crop, axis=-1), axis=0))

In [ ]:
# verify they produce the same results

In [54]:
np.array_equal(patch_crop_encoded_zzz[0], patch_crop_encoded_zzz2[0])

True

In [55]:
np.array_equal(patch_crop_encoded_zzz[1], patch_crop_encoded_zzz2[1])

True

In [56]:
# should be false beacause of randomness
np.array_equal(patch_crop_encoded_zzz[2], patch_crop_encoded_zzz2[2])

False

In [58]:
vae_encoder.save_weights('weights/vae_encoder_' + model_version + '.h5')